In [3]:
import numpy
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import scipy.sparse as ss
from scipy.sparse import csr_matrix
print(ad.__version__)

0.10.9


/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


#### read h5 matrices

In [4]:
raw_h5_path = '../01_Daten/raw_feature_bc_matrix.h5'
filtered_h5_path = '../01_Daten/filtered_feature_bc_matrix.h5'

# sc.read_10x_h5(filename, *, genome=None, gex_only=True, backup_url=None)
raw_adata = sc.read_10x_h5(raw_h5_path, gex_only=True)
filtered_adata = sc.read_10x_h5(filtered_h5_path, gex_only=True)

/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/oskarhaupt/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


#### make var_names unique

In [5]:
raw_adata.var_names_make_unique()
filtered_adata.var_names_make_unique()
#die barcodes wurden in der filtered reduziert

In [6]:
filtered_adata

AnnData object with n_obs × n_vars = 2701 × 22040
    var: 'gene_ids', 'feature_types', 'genome', 'interval'

#### structure of AnnData

In [7]:
# obs = barcodes/cells, var = genes
#raw_adata.layers["log_transformed"] = np.log1p(raw_adata.X)
# dauert viel zu lang !!!!!!!
#raw_adata.to_df(layer="log_transformed")

In [23]:
cell_barcodes = raw_adata.obs_names[:5]  # First 5 cell barcodes
gene_names = raw_adata.var_names         # All gene names
umi_counts = raw_adata.X[:5, :]          # UMI counts for the first 5 cells

if hasattr(umi_counts, "todense"):  # Check if `umi_counts` is sparse
    print(umi_counts)
    umi_counts = umi_counts.todense()
    print("after: ", umi_counts)

umi_df = pd.DataFrame(umi_counts, index=cell_barcodes, columns=gene_names)

# Display the first 5 rows
print(umi_df.head().values)

#print values that are not 1.0


  (0, 1768)	1.0
  (0, 2699)	1.0
  (0, 9238)	1.0
  (0, 17302)	1.0
  (0, 18332)	1.0
  (2, 979)	1.0
  (2, 1151)	1.0
  (2, 1595)	1.0
  (2, 2159)	1.0
  (2, 3129)	1.0
  (2, 3136)	1.0
  (2, 3958)	1.0
  (2, 5480)	1.0
  (2, 6201)	1.0
  (2, 6617)	1.0
  (2, 7451)	1.0
  (2, 8030)	1.0
  (2, 9439)	1.0
  (2, 9761)	1.0
  (2, 11012)	1.0
  (2, 11112)	1.0
  (2, 15055)	1.0
  (2, 15938)	1.0
  (2, 16738)	1.0
  (2, 17393)	1.0
  (2, 17515)	1.0
  (2, 19846)	1.0
  (2, 19939)	1.0
  (2, 20061)	1.0
  (2, 20368)	1.0
  (3, 88)	1.0
  (3, 366)	1.0
  (3, 2328)	1.0
  (3, 2806)	1.0
  (3, 2993)	1.0
  (3, 5355)	1.0
  (3, 6694)	1.0
  (3, 7489)	1.0
  (3, 9232)	1.0
  (3, 11976)	1.0
  (3, 15804)	1.0
  (3, 20472)	1.0
  (4, 18251)	1.0
after:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
print(raw_adata.X[0].data) #gives the non zero values
print(raw_adata.X[0].indices) #gives the gene position of the non zero values

[1. 1. 1. 1. 1.]
[ 1768  2699  9238 17302 18332]


In [10]:
rows, cols = raw_adata.X[0].nonzero()# Get row and column indices of non-zero elements
for row in rows:
    print(row)
for row, col in zip(rows, cols):
    value = raw_adata.X[row, col]  # Access the value
    print(f"Row {row}, Column {col}, Value {value}")


0
0
0
0
0
Row 0, Column 1768, Value 1.0
Row 0, Column 2699, Value 1.0
Row 0, Column 9238, Value 1.0
Row 0, Column 17302, Value 1.0
Row 0, Column 18332, Value 1.0


In [11]:
raw_adata.var_names[0]

'ALDH1A1'

In [12]:
print(raw_adata.X[0])

  (0, 1768)	1.0
  (0, 2699)	1.0
  (0, 9238)	1.0
  (0, 17302)	1.0
  (0, 18332)	1.0


In [13]:
print(len(gene_names)) 

22040


In [14]:
# Step 1: Use `find` to extract row indices, column indices, and values for all non-zero entries
rows, cols, values = ss.find(raw_adata.X)
max_count = values.max()
max_index = values.argmax()
max_row, max_col = rows[max_index], cols[max_index]

print("Maximum UMI count:", max_count)
print("Location of maximum count: (Row/cell-barcode, Column/gene) =", (max_row, max_col))

Maximum UMI count: 5305.0
Location of maximum count: (Row/cell-barcode, Column/gene) = (403321, 11939)


In [15]:
type(max_count)

numpy.float32

In [16]:
a = 15.0
if(a%1==0):
    print("check")

check


In [17]:
type(umi_counts)
print(umi_counts)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
float_count = 0
if isinstance(filtered_adata.X, csr_matrix):
    for value in filtered_adata.X.data:
        if isinstance(value, np.float32):
            raise ValueError(f"Error: Found a float value {value} in the matrix, which is not allowed.")
            # OR 
            float_count +=1          
else:
    print("The provided matrix is not a CSR matrix.")
    
print("float_count: ", float_count)

response = input(f"Warning: Found {float_count} float values in the matrix. Do you want to continue? (yes/no): ")
if response.lower() != 'yes':
    print("Exiting process.")
    #break



ValueError: Error: Found a float value 1.0 in the matrix, which is not allowed.

In [ ]:
def newton(f, f_ableitung, x0, iterationen=4):
    approx_list = [x0]
    for _ in range(iterationen):
        x_naechste = x0 - f(x0) / f_ableitung(x0)
        approx_list.append(x_naechste)
        x0 = x_naechste
    return approx_list

f = lambda x: x**2 - x - 1
f_ableitung = lambda x: 2*x - 1

x0_1 = -2
x0_2 = -4

approx_x0_1 = newton(f, f_ableitung, x0_1)
approx_x0_2 = newton(f, f_ableitung, x0_2)

print("Approx. für x0 = -2:")
for i, x in enumerate(approx_x0_1):
    print(f"x{i} = {x}")

print("\nApprox. für x0 = -4:")
for i, x in enumerate(approx_x0_2):
    print(f"x{i} = {x}")

In [ ]:
# first trys with muon
import numpy as np
import pandas as pd
import scanpy as sc

from matplotlib import pyplot as plt
import seaborn as sns
import muon as mu

/opt/anaconda3/envs/BIH_first-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
raw_h5_path = '../01_Daten/raw_feature_bc_matrix.h5'
mdata = mu.read_10x_h5("raw_h5_path")
mdata

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'raw_h5_path', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
mu.pp.intersect_obs(mdata)